# Train classifier to estimate analysis efficiency vs gen variable

- Actual training code in train.py
- Classifier based on sklearn. Default is GradientBoostedClassifier, 
    but can be specified at run time.

## Load libraries

In [1]:
import train as tn
#reload(tn)

import plotting
reload(plotting)

import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')
%matplotlib inline

import numpy as np

from pprint import pprint

import os
import json
import importlib

import util as ut
reload(ut)

Welcome to ROOTaaS 6.06/08


<module 'util' from '/mnt/t3nfs01/data01/shome/jandrejk/higgs_model_dep/util.pyc'>

## Instantiate helper class

Data are read from ROOT trees and converted into pandas data frames.  
The loading function makes sure that all the needed columns have been read from the trees, otherwise it rebilds the data frame.


#### In the following cell the parameters are set up. In this case a new classifier will be initiated with the name effGenVarClass and stored in the directory "classifiers". The training data will be read from a root tree located in dataDir. The branches of gen and reco events are the default ones (look at the help function for detailed information).

In [2]:
ut.defaultParameters(dataDir="/mnt/t3nfs01/data01/shome/jandrejk/higgs_model_dep/data", 
                     classifiers=['class'],#,'recoPt','recoNjets2p5'],
                          load = False,
                     inputName = "class_tnevts_etaLead_etaSublead",
                       outName = "class_tnevts_etaLead_etaSublead_out",
                        outDir = './classifiers',
                     # InputDir = './classifiers',
                    #defineBins = { 'recoPt' : dict(boundaries=[0.,15.,30.,45.,85.,125.,200.,350.,10000.],overflow=False), # do not add overflow automatically
                    #              'recoNjets2p5' : dict(boundaries=[-0.5,0.5,1.5,2.5,3.5,100.],overflow=False),
                    #              #'genPt' : dict(boundaries=[0.,15.,30.,45.,85.,125.,200.,350.,10000.],overflow=False), # do not add overflow automatically
                    #              #'genNjets2p5' : dict(boundaries=[-0.5,0.5,1.5,2.5,3.5,100.],overflow=False)
                    #             },
                     dataFiles=[(0,'output_InsideAcceptance_ggF_125.root'),                   
                                (1,'output_InsideAcceptance_ttH_125.root'),                     
                                (2,'output_InsideAcceptance_VBF_125.root'),                            
                                (3,'output_InsideAcceptance_VH_125.root')
                               ]
                    )



In case you need help or further explanation you can execute the help function:

#### We want to use machine learning techniques for classification. For that we need to specify the classifier we want to use like here the GradientBoostingClassifier from sklearn.ensamble. Furthermore the parameters of the used classifier have to be set, e.g. the number of training events, max. tree depth, learning rate and so forth.

In [3]:
ut.params["class"] = [ "sklearn.ensemble.GradientBoostingClassifier", 
                      dict(trainevts= 100,
                           max_depth=5,learning_rate=0.2,n_estimators=200,
                        min_weight_fraction_leaf=1e-3)
]


"""ut.params['recoPt'] = ["xgboost.sklearn.XGBClassifier",
                       { "Xbr" : ["genPt","absGenRapidity"],
                        "trainevts" : 10, 
                        "max_depth" : 5,"learning_rate" : 0.1,
                        "n_estimators" : 500,"min_child_weight" : 1e-5,
                        "nthread" : 1}]
"""
"""
,
                        "cvoptimize" : True,"cv_params_grid" : { "max_depth" : [5, 7, 10],
                                                                 "learning_rate" : [0.05, 0.1, 0.2], 
                                                                 "n_estimators" : [250,500,700,1000],
                                                                 "min_child_weight" : [1e-4, 5e-4, 1e-3],
                                                                 "subsample" : [0.1, 0.2, 0.5, 1.]}, 
                        "cv_nfolds" : 5, "cv_niter" : 5,      
"""

"""ut.params['recoNjets2p5'] =  ["xgboost.sklearn.XGBClassifier",
                        { "Xbr" : ["genJet2p5Pt0", "genJet2p5Rapidity0",
                                    "genJet2p5Pt1", "genJet2p5Rapidity1",
                                    "genJet2p5Pt2", "genJet2p5Rapidity2",
                                    "genJet2p5Pt3", "genJet2p5Rapidity3",
                                    "genPt","absGenRapidity"
                                ],
            "trainevts" : 10, "max_depth" : 7,
            "learning_rate" : 0.1,"n_estimators" : 500,
            "min_child_weight" : 1e-5,
            "nthread" : 1 }]
"""

'ut.params[\'recoNjets2p5\'] =  ["xgboost.sklearn.XGBClassifier",\n                        { "Xbr" : ["genJet2p5Pt0", "genJet2p5Rapidity0",\n                                    "genJet2p5Pt1", "genJet2p5Rapidity1",\n                                    "genJet2p5Pt2", "genJet2p5Rapidity2",\n                                    "genJet2p5Pt3", "genJet2p5Rapidity3",\n                                    "genPt","absGenRapidity"\n                                ],\n            "trainevts" : 10, "max_depth" : 7,\n            "learning_rate" : 0.1,"n_estimators" : 500,\n            "min_child_weight" : 1e-5,\n            "nthread" : 1 }]\n'

In [4]:
#ut.params

$\color{red}{\text{changes in class become also active without ut.setParams(). }}$

In [5]:
#set the parameters "class" defined above
"""
? discuss the function setParams() with Pasquale
"""
ut.setParams()

entered config files named my_train_config
io.json
load some params
reading io.json


In [6]:
%time effFitter = ut.loadOrMake()

Create object with the name class_tnevts_etaLead_etaSublead and the following paramters 
The number of selected events are 572356
The number of selected events are 766839
The number of selected events are 1448533
The number of selected events are 1583887
shuffling dataset
defining bins
CPU times: user 3.52 s, sys: 388 ms, total: 3.9 s
Wall time: 5.57 s


## Train classifiers

# $\color{red}{\text{What happens in the bellow 2 cells?}}$

In [7]:
if ut.params.get("setupJoblib",True):
    ut.setupJoblib(ipp_profile=ut.params.get('ippProfile','short'))

Waiting for connection file: ~/.ipython/profile_short/security/ipcontroller-client.json


IOError: Connection file '~/.ipython/profile_short/security/ipcontroller-client.json' not found.
You have attempted to connect to an IPython Cluster but no Controller could be found.
Please double-check your configuration and ensure that a cluster is running.

###  make sure that the trained classifers have been evaluated

In [8]:
ut.runEvaluation(effFitter)

Index([u'absweight', u'class', u'genJet2p5Pt0', u'genJet2p5Pt1',
       u'genJet2p5Pt2', u'genJet2p5Pt3', u'genJet2p5Rapidity0',
       u'genJet2p5Rapidity1', u'genJet2p5Rapidity2', u'genJet2p5Rapidity3',
       u'genLeadEta', u'genNjets2p5', u'genPt', u'genRapidity',
       u'genSubleadEta', u'recoNjets2p5', u'recoPt', u'recoRapidity',
       u'weight', u'proc', u'absGenRapidity', u'absGenJet2p5Rapidity0',
       u'absGenJet2p5Rapidity1', u'absGenJet2p5Rapidity2',
       u'absGenJet2p5Rapidity3'],
      dtype='object')


### Run the actual training

In [9]:
#reload(tn)
%time ut.runTraining(effFitter)

We need to train the following classifiers class
Fitting class
<class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
{'n_estimators': 200, 'learning_rate': 0.2, 'min_weight_fraction_leaf': 0.001, 'max_depth': 5, 'trainevts': 100}
['genPt', 'absGenRapidity', 'genLeadEta', 'genSubleadEta']
class
cvoptimize False
((100, 4), 400)
((100,), 100)
((100,), 100)
[[  4.78662491e+01   2.79784262e-01   7.23288655e-02   7.06442535e-01]
 [  3.81379356e+01   2.39786124e+00  -2.38502049e+00  -2.41999459e+00]
 [  1.92517136e+02   1.06277466e-02   3.24289680e-01  -5.44572234e-01]]
[ 0 -1  0]
[  3.46337474e-05   2.94910307e-04   4.54608926e-05]
CPU times: user 14.4 s, sys: 150 ms, total: 14.6 s
Wall time: 14.6 s


## Save the output

In [10]:
#reload(tn)
%time tn.IO.save(effFitter)

./Train_Configurable_Refactor_higgs_model_dep_2255084.pkl.gz
CPU times: user 9.95 s, sys: 237 ms, total: 10.2 s
Wall time: 10.9 s
